In [1]:
from sqlalchemy import create_engine
from db_constants import Base
from sqlalchemy import func
import pandas as pd
from models import ParentPair, ChildPair
from sqlalchemy.orm import sessionmaker
from sqlalchemy import String
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_32618/249799025.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
results = DataLoader().get_latest_data()
df = pd.DataFrame(results)

# Define a mapping dictionary
mapping_dict = {'1': 'A/B', '2': 'A/C', '3': 'B/C'}

# Replace the numerical suffixes in column names
df.columns = df.columns.to_series().replace(mapping_dict, regex=True)

# Display the modified DataFrame
pd.set_option('display.max_rows', None)
columns_to_check = ['sub_pair_B/C', 'sub_pair_A/B', 'sub_pair_A/C', 'bid_price_B/C', 'bid_price_A/B', 'bid_price_A/C',
                     'ask_price_B/C', 'ask_price_A/B', 'ask_price_A/C']

df[columns_to_check] = df[columns_to_check].replace('0', pd.NA)
df = df.dropna(subset=columns_to_check)

price_columns = ['bid_price_B/C', 'bid_price_A/B', 'bid_price_A/C', 
                  'ask_price_B/C', 'ask_price_A/B', 'ask_price_A/C']

df[price_columns] = df[price_columns].astype(float)
df['price_A/C_2'] = df['ask_price_A/B'] * df['ask_price_B/C']
df['is_s<__1'] = df['ask_price_A/B'] * df['ask_price_B/C'] * (1 / df['ask_price_A/C'])

df['is_g>__1'] = df['bid_price_A/B'] * df['bid_price_B/C'] * (1 / df['bid_price_A/C'])
df['s<1_diff'] = 1 - df['is_s<__1']
df['g>1_diff'] = df['is_g>__1'] - 1
df['diff'] = df[['s<1_diff', 'g>1_diff']].max(axis=1)
df['chosen_diff'] = df[['s<1_diff', 'g>1_diff']].idxmax(axis=1)
# Sort by the maximum diff value
df.loc[(df['s<1_diff'] < 0) & (df['g>1_diff'] < 0), 'chosen_diff'] = '-'
sorted_df = df.sort_values(by='diff', ascending=False)
print(len(sorted_df))
display(sorted_df)


8


,parent_pair,sub_pair_A/B,bid_price_A/B,ask_price_A/B,sub_pair_A/C,bid_price_A/C,ask_price_A/C,sub_pair_B/C,bid_price_B/C,ask_price_B/C,price_A/C_2,is_s<__1,is_g>__1,s<1_diff,g>1_diff,diff,chosen_diff
5,ETH-BTC-ZAR,ETHBTC,0.05352,0.05353,ETHZAR,44237.00,44319.00,BTCZAR,8.275240e+05,8.276670e+05,4.430501e+04,0.999684,1.001177,0.000316,0.001177,0.001177,g>1_diff
6,ETH-BTC-BIDR,ETHBTC,0.05352,0.05353,ETHBIDR,36025286.00,36053831.00,BTCBIDR,6.731847e+08,6.736371e+08,3.605979e+07,1.000165,1.000099,-0.000165,0.000099,0.000099,g>1_diff
0,ETH-BTC-USDT,ETHBTC,0.05352,0.05353,ETHUSDT,2291.14,2291.15,BTCUSDT,4.280830e+04,4.280831e+04,2.291529e+03,1.000165,0.999983,-0.000165,-0.000017,-0.000017,-
2,ETH-BTC-USDC,ETHBTC,0.05352,0.05353,ETHUSDC,2290.32,2290.33,BTCUSDC,4.279222e+04,4.279223e+04,2.290668e+03,1.000148,0.999965,-0.000148,-0.000035,-0.000035,-
3,ETH-BTC-TRY,ETHBTC,0.05352,0.05353,ETHTRY,70531.00,70539.00,BTCTRY,1.317789e+06,1.317848e+06,7.054440e+04,1.000077,0.999958,-0.000077,-0.000042,-0.000042,-
1,ETH-BTC-TUSD,ETHBTC,0.05352,0.05353,ETHTUSD,2322.52,2322.67,BTCTUSD,4.339279e+04,4.339280e+04,2.322817e+03,1.000063,0.999941,-0.000063,-0.000059,-0.000059,-
4,ETH-BTC-EUR,ETHBTC,0.05352,0.05353,ETHEUR,2125.23,2125.24,BTCEUR,3.970521e+04,3.970522e+04,2.125420e+03,1.000085,0.999903,-0.000085,-0.000097,-0.000085,-
7,ETH-BTC-DAI,ETHBTC,0.05352,0.05353,ETHDAI,2290.36,2290.38,BTCDAI,4.279081e+04,4.279698e+04,2.290922e+03,1.000237,0.999914,-0.000237,-0.000086,-0.000086,-
